# **Извлечение ключевых слов**

В качестве корпуса я решила взять статьи из журнала [Cosmopolitan](https://www.cosmo.ru/). В рамках домашнего задания мне бы хотелось отследить влияние длины текста, поэтому в корпус попали и относительно длинные тексты в 1000 слов, средние по длине - по 500 и совсем короткие тексты. В результате в [корпус](https://github.com/psaleksandrova/NLP_2020/blob/main/HW1/cosmo.csv) вошло 7 статей.

Ключевые слова хранятся в виде тегов. Тем не менее, теги, выбранные авторами статьи, в некоторых случаях достаточно общими, что я и постаралась исправить в своей ручной разметке, сделав теги более специфичными. Кажется, что теги для статей используются в первую очередь как способ навигации, а потому совершенно нормально, что они отражают общую идею статьи, несмотря на то, что выбранные для некоторых тегов слова даже не встречаются в тексте.

In [165]:
import pandas as pd

In [166]:
df = pd.read_csv('https://raw.githubusercontent.com/psaleksandrova/NLP_2020/main/HW1/cosmo.csv?token=AHTHNY3322ACLGJ2X4EQDSK74JVGG')
df['text_len'] = df['text'].apply(lambda x: len(x.split()))
df

,text,keywords,my_keywords,text_len
0,40-летняя теледива сняла провокационные кадры ...,"Ким Кардашьян, Канье Уэст, звездные семьи, сем...","Ким Кардашьян, наряд, Канье Уэст, развод",445
1,Для кого мы придумали Школу женского здоровья?...,"женское здоровье, аборт, менструация, беременн...","женщины, школа женского здоровья, медицина, бе...",513
2,"Каждой маме знакомо это чувство: кажется, что ...","женщины, дети, материнство, психология, что де...","женщины, материнство, дети, выгорание, усталос...",1061
3,"Как побороть лень, апатию и усталость?\r\nБыст...","эмоциональное выгорание, лень, борьба с устало...","хроническая усталость, лень, апатия, мотивация...",1084
4,38-летний иллюзионист Сергей Сафронов был со с...,"Ольга Бузова, Сергей Сафронов, скандалы, Битва...","Алина Вердиш, Сергей Сафронов, победа, ТНТ, Би...",352
5,41-летняя Виктория Боня выступила против массо...,"Виктория Боня, коронавирус, COVID-19, пандемия","Виктория Боня, коронавирус, прививка, ВОЗ",229
6,При поисковом запросе «диета Протасова» Google...,"здоровье, похудение, рецепты, диета","похудение, диета, Ким Протасов, Ганна",201


In [167]:
sum(df['text_len'])

3885

Разберемся с эталонной разметкой

In [168]:
[list(set(df.keywords[i].split(', ')) & set(df.my_keywords[i].split(', '))) for i in range(len(df))]

[['Ким Кардашьян', 'Канье Уэст'],
 ['беременность'],
 ['дети', 'материнство', 'выгорание', 'женщины'],
 ['лень'],
 ['Битва экстрасенсов', 'Сергей Сафронов'],
 ['Виктория Боня', 'коронавирус'],
 ['похудение', 'диета']]

Кажется, что вследствие низкой размерности пересечений списков ключевых слов более правильным будет использовать их объединение.

In [169]:
keys = [list(set(df.keywords[i].split(', ')) | set(df.my_keywords[i].split(', '))) for i in range(len(df))]
keys

[['семейство Кардашьян',
  'наряд',
  'звездные семьи',
  'Ким Кардашьян',
  'развод',
  'Канье Уэст',
  'корректирующее белье'],
 ['беременность',
  'женское здоровье',
  'аборт',
  'менструация',
  'медицина',
  'школа женского здоровья',
  'гормональные изменения',
  'женщины'],
 ['материнство',
  'выгорание',
  'дети',
  'помощь',
  'что делать',
  'усталость',
  'психология',
  'женщины'],
 ['эмоциональное выгорание',
  'хроническая усталость',
  'мотивация',
  '\r\nботе',
  'лень',
  'отдых',
  'борьба с усталостью',
  'апатия',
  'работа'],
 ['ТНТ',
  'победа',
  'Битва экстрасенсов',
  'скандалы',
  'Ольга Бузова',
  'Алина Вердиш',
  'Сергей Сафронов'],
 ['ВОЗ', 'коронавирус', 'COVID-19', 'Виктория Боня', 'пандемия', 'прививка'],
 ['похудение', 'рецепты', 'Ким Протасов', 'здоровье', 'диета', 'Ганна']]

In [170]:
df['etalon_keywords'] = keys

## Перейдём к самому извлечению ключевых слов

In [171]:
!pip install pymorphy2
!pip install razdel

In [172]:
import re
import pymorphy2
import nltk
nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords
from razdel import tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [173]:
PUNCT = punctuation + '«»--—…“”*№–'
MORPH = pymorphy2.MorphAnalyzer()
STOP = stopwords.words('russian')

In [174]:
def preproc_text(text):
    clean_text = re.sub(r'<.*>', '', text)
    clean_text = ' '.join([word.strip(PUNCT) for word in clean_text.lower().split() if word not in STOP])
    tokens = list(tokenize(clean_text))
    lemmas = ' '.join([MORPH.parse(word.text)[0].normal_form for word in tokens])
    return lemmas

Проверим функцию:

In [175]:
preproc_text('Это секрет, но я люблю шоколадки :) И сайт <html://смешные-коты.рф>!!!')

'это секрет любить шоколадка сайт'

Теперь смело применим её ко всем нашим текстам датасета

In [176]:
lemmas = [preproc_text(text) for text in df.text]

## RAKE

In [177]:
!pip install python-rake

In [178]:
import RAKE

In [179]:
rake = RAKE.Rake(STOP)

In [180]:
rake.run(lemmas[3], maxWords=3, minFrequency=1)

[('напротив мешать', 4.0),
 ('это происходить', 4.0),
 ('право ошибка', 4.0),
 ('ошибаться', 1.0),
 ('поддержать', 1.0)]

Исключим слово "это" из списка стоп-слов

In [181]:
STOP.append("это")
rake = RAKE.Rake(STOP)

In [182]:
keywords_rake = [rake.run(text, maxWords=3, minFrequency=1) for text in lemmas]

In [183]:
df['keywords_rake'] = keywords_rake

In [184]:
df['keywords_rake'] = df['keywords_rake'].apply(lambda x: [i[0] for i in x])

## TextRank

In [185]:
!pip install summa

In [186]:
from summa import keywords

In [187]:
keywords.keywords(lemmas[0], language='russian', additional_stopwords=STOP, scores=True)

[('свой', 0.36476945597253496),
 ('ким кардашьян', 0.22550169046543894),
 ('который', 0.22160408035432788),
 ('спина', 0.19138356509144538),
 ('оставаться', 0.15097391196958726),
 ('наряд звезда', 0.14927020195326218),
 ('время', 0.1444509409391609),
 ('белый платье', 0.1259056747512247),
 ('фигура бодить', 0.12206072353096112),
 ('жить', 0.11887647355578929),
 ('канье уэст', 0.1114853104082413),
 ('жизнь', 0.10387029653134708),
 ('корректировать бельё', 0.0955159959898951),
 ('примерить', 0.09235230548066083),
 ('роскошный', 0.09169549257578868),
 ('роскошно', 0.09169549257578868),
 ('часть', 0.08942600694181793),
 ('сообщать', 0.08727311859739903),
 ('пара', 0.08528064947739136),
 ('стараться', 0.08053425948297199),
 ('окружение', 0.07923988822797377),
 ('приклеивать', 0.0776862225709352),
 ('проблема', 0.07621854154938469),
 ('семейство', 0.07291922744888442),
 ('изгиб', 0.07276184475196058),
 ('оба казаться', 0.07266010659165775),
 ('летний', 0.07060435590183409),
 ('столько', 0.06

Также исключим слово "который"

In [188]:
STOP.append("который")

In [189]:
keywords_textrank = [keywords.keywords(text, language='russian', additional_stopwords=STOP, scores=True) for text in lemmas]
df['keywords_textrank'] = keywords_textrank

In [190]:
df['keywords_textrank'] = df['keywords_textrank'].apply(lambda x: [i[0] for i in x])

## TF-IDF

In [191]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [192]:
vectorizer = TfidfVectorizer(stop_words=STOP)

Пример

In [193]:
X = vectorizer.fit_transform([lemmas[0]])
feature_names = np.array(vectorizer.get_feature_names())
denselist = np.array(X.todense())
indices = denselist.argsort()[0]
feature_names[indices[::-1][:10]]

array(['свой', 'ким', 'канье', 'платье', 'спина', 'звезда', 'кардашьян',
       'время', 'бодить', 'корректировать'], dtype='<U18')

Возьмём по 10 слов для каждого текста

In [209]:
keywords_tfidf = []

for sent in lemmas:
    X = vectorizer.fit_transform([sent])
    feature_names = np.array(vectorizer.get_feature_names())
    denselist = np.array(X.todense())
    indices = denselist.argsort()[0]
    keywords_tfidf.append(feature_names[indices[::-1][:10]])

In [217]:
df['keywords_tfidf'] = keywords_tfidf

In [218]:
df['keywords_textrank'] = df['keywords_textrank'].apply(lambda x: list(x))

# Морфологические/синтаксические шаблоны для ключевых слов и фраз

Попробуем использовать шаблоны Adj+Noun и Noun+Noun, помимо просто Noun

In [197]:
TEMPLATES = [['NOUN'], ['NOUN', 'NOUN'], ['ADJF', 'NOUN']]

In [198]:
def pos_keywords(keywords):
    pos_tags = [[MORPH.parse(w)[0].tag.POS for w in words.split()] for words in keywords]
    res = [words for i, words in enumerate(keywords) if pos_tags[i] in TEMPLATES]
    return res

Проверим функцию

In [199]:
pos_keywords(['хотеть спать', 'красивый наряд', 'кот', 'кот грустно', 'Ким Кардашьян'])

['красивый наряд', 'кот', 'Ким Кардашьян']

In [220]:
df['keywords_rake_POS'] = df['keywords_rake'].apply(lambda x: pos_keywords(x))
df['keywords_textrank_POS'] = df['keywords_textrank'].apply(lambda x: pos_keywords(x))
df['keywords_tfidf_POS'] = df['keywords_tfidf'].apply(lambda x: pos_keywords(x))

In [221]:
df

,text,keywords,my_keywords,text_len,etalon_keywords,keywords_rake,keywords_textrank,keywords_tfidf,keywords_rake_POS,keywords_textrank_POS,keywords_tfidf_POS
0,40-летняя теледива сняла провокационные кадры ...,"Ким Кардашьян, Канье Уэст, звездные семьи, сем...","Ким Кардашьян, наряд, Канье Уэст, развод",445,"[семейство кардашьян, наряд, звёздный семья, к...","[столько секс, делать ребёнок, казаться беспок...","[свой, ким кардашьян, спина, оставаться, время...","[свой, ким, канье, платье, спина, звезда, кард...",[],"[ким кардашьян, спина, время, наряд звезда, бе...","[ким, канье, платье, спина, звезда, кардашьян,..."
1,Для кого мы придумали Школу женского здоровья?...,"женское здоровье, аборт, менструация, беременн...","женщины, школа женского здоровья, медицина, бе...",513,"[беременность, женский здоровье, аборт, менстр...",[поддаваться коррекция знать],"[здоровый, школа женский здоровье, забота свой...","[здоровье, женский, свой, женщина, репродуктив...",[],"[девушка женщина, задача, возраст заболевание,...","[здоровье, женщина, задача, партнёр, программа..."
2,"Каждой маме знакомо это чувство: кажется, что ...","женщины, дети, материнство, психология, что де...","женщины, материнство, дети, выгорание, усталос...",1061,"[материнство, выгорание, ребёнок, помощь, дела...","[каждый мама знакомый, далее желательно труд, ...","[справляться ребёнок собственный жизнь, сила, ...","[ребёнок, свой, мать, твой, время, сила, женщи...","[дело проблема, дело, выгорание, норма, мать]","[сила, хороший мать, женщина, мочь, уход, особ...","[ребёнок, мать, время, сила, женщина, дело, мочь]"
3,"Как побороть лень, апатию и усталость?\r\nБыст...","эмоциональное выгорание, лень, борьба с устало...","хроническая усталость, лень, апатия, мотивация...",1084,"[эмоциональный выгорание, хронический усталост...","[напротив мешать, право ошибка, происходить, о...","[свой, задача, помогать, часто, дело, время, а...","[усталость, задача, свой, дело, помогать, част...",[право ошибка],"[задача, дело, время, работа, перфекционизм, с...","[усталость, задача, дело, работа, время, апати..."
4,38-летний иллюзионист Сергей Сафронов был со с...,"Ольга Бузова, Сергей Сафронов, скандалы, Битва...","Алина Вердиш, Сергей Сафронов, победа, ТНТ, Би...",352,"[тнт, победа, битва экстрасенс, скандал, ольга...",[воротников ведьма],"[алин, иллюзионист сергей сафронов, алина верд...","[сафронов, экстрасенс, сергей, шоу, алина, илл...",[воротников ведьма],"[алин, алина вердиш, тысяча, брат, черта, чёрт...","[сафронов, экстрасенс, сергей, шоу, алина, илл..."
5,41-летняя Виктория Боня выступила против массо...,"Виктория Боня, коронавирус, COVID-19, пандемия","Виктория Боня, коронавирус, прививка, ВОЗ",229,"[воз, коронавирус, covid-19, виктория бонить, ...","[который принадлежать воз, приходить полиция и...","[виктория бонить, прививка, считать, вакцина, ...","[бонить, прививка, виктория, говорить, считать...","[вакцинация, пофига]","[виктория бонить, прививка, вакцина, пандемия]","[бонить, прививка, виктория, коронавирус, вакц..."
6,При поисковом запросе «диета Протасова» Google...,"здоровье, похудение, рецепты, диета","похудение, диета, Ким Протасов, Ганна",201,"[похудение, рецепт, ким протас, здоровье, диет...","[равно придумать, сколько год]","[протас, диета протасов, ганна, популярный, по...","[диета, протасов, ким, ганна, имя, год, протас...",[],"[протас, диета протасов, ганна, популярность, ...","[диета, протасов, ким, ганна, имя, год, протас..."


Теперь, к сожалению из RAKE пропали почти все слова( Но, кажется, что их было мало и в самом начале, поэтому стоит внимательнее изучить, что произошло, при дальнейшем анализе.

# Посчитаем нужные метрики

Эталонные ключевые слова для корректности подсчёта необходимо нормализовать тем же методом, который был применён для самиъ текстов. Помимо того, что слова в эталонных тегах не стоят в начальной форме, в ключевые попали нестандартные слова, как, например, фамилия Боня, которая нормализуется в бонить. При одинаковой нормализации и эталонного набора ключевых слов проблем в расхождении не будет.

In [202]:
df['etalon_keywords'] = df['etalon_keywords'].apply(lambda x: [preproc_text(text) for text in x])

In [203]:
df['etalon_keywords']

0    [семейство кардашьян, наряд, звёздный семья, к...
1    [беременность, женский здоровье, аборт, менстр...
2    [материнство, выгорание, ребёнок, помощь, дела...
3    [эмоциональный выгорание, хронический усталост...
4    [тнт, победа, битва экстрасенс, скандал, ольга...
5    [воз, коронавирус, covid-19, виктория бонить, ...
6    [похудение, рецепт, ким протас, здоровье, диет...
Name: etalon_keywords, dtype: object

Также добавим отдельный столбик для применения шаблона к нашему эталону:

In [239]:
df['etalon_keywords_POS'] = df['etalon_keywords'].apply(lambda x: pos_keywords(x))

In [204]:
def count_precision(etalon_keywords, pred):
    return np.intersect1d(etalon_keywords, pred).shape[0] / len(pred) if pred else 0

def count_recall(etalon_keywords, pred):
    return np.intersect1d(etalon_keywords, pred).shape[0] / len(etalon_keywords) if etalon_keywords else 0

def count_f1_score(etalon_keywords, pred):
    precision = count_precision(etalon_keywords, pred)
    recall = count_recall(etalon_keywords, pred)
    return 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0

In [222]:
df_results = pd.DataFrame(columns=['method', 'precision', 'recall', 'f1 score'])

In [230]:
METHODS = ['keywords_rake', 'keywords_rake_POS', 'keywords_textrank', 
           'keywords_textrank_POS', 'keywords_tfidf', 'keywords_tfidf_POS']

In [240]:
for i, method in enumerate(METHODS):
    etalon_keywords = 'etalon_keywords_POS' if method.endswith('POS') else 'etalon_keywords'
    precision = []
    recall = []
    f1_score = []

    for j in range(len(lemmas)):
        precision.append(count_precision(df[etalon_keywords][j], list(df[method][j])))
        recall.append(count_recall(df[etalon_keywords][j], list(df[method][j])))
        f1_score.append(count_f1_score(df[etalon_keywords][j], list(df[method][j])))
    
    df_results.loc[i] = [method, np.array(precision).mean(), 
                         np.array(recall).mean(), np.array(f1_score).mean()]

In [241]:
df_results

,method,precision,recall,f1 score
0,keywords_rake,0.015873,0.035714,0.021978
1,keywords_rake_POS,0.028571,0.020408,0.023810
2,keywords_textrank,0.081240,0.354025,0.128203
3,keywords_textrank_POS,0.205650,0.338549,0.232613
4,keywords_tfidf,0.142857,0.204365,0.166980
5,keywords_tfidf_POS,0.195011,0.226304,0.206463


# Анализ результатов

Результаты оказываются сравнительно низкими, однако видна тенденция улучшения в сторону методов, использующих морфосинтаксические шаблоны. Таким образом, первым шагом к улучшению качества алгоритмов может стать донастройка таких шаблонов.

Наиболее низкие результаты наблюдается для алгоритма RAKE, в который включаются лишние слова, не относящиеся к конкретному тексту. В первую очередь это глаголы типа "казаться", что привело к тому, что в результате применения морфосинтаксических шаблонов для некоторых текстов ключевых слов не осталось совсем). Можно попробовать выделить отдельный список стоп-слов, специфичных для всех текстов из данного ресурса. При таком подходе в результате работы алгоритма останутся только действительно специфичные для конкретного текста слова.

Кажется, что основную проблему вызвала специфичность текстов статей. Большинство текстов посвящается конкретным личностям, но для избежания повторений в тексте используются различные референции к персонажу: имя + фамилия, вариации имени, род деятельности человека и т.д., эквивалентность которых игнорируется алгоритмом. Можно было бы пересмотреть эталонную разметку исходя из кореферентности и добавить все возможные слова и словосочетания, однако кажется, что более правильным решением является подключение инструментов для разрешения кореферентности.

Ещё можно было бы подробнее изучить специфику жанра выбранных текстов. Кажется, что короткие статьи в интернет-журналах/газетах тяготеют к вынесению основной информации в начало текста, чтобы привлечь внимание читателя и сразу же удержать его. А потому можно было бы попробовать присваивать больший вес словам из начала текста исходя из какой-либо эвристики.
